# Energy balance checks
This notebook summarizes SUMMA's internal enthalpy balance checks.

## Theory
The general equation used is (Eq. 1):

&nbsp;&nbsp;&nbsp;&nbsp; 
$\frac{\delta H^\Omega}{\delta t} = - \frac{\delta F^\Omega}{\delta z} + H_{sink}^\Omega,$

where $H_{sink}^\Omega$ is assumed to be `0`. Normalizing by depth `z`:

&nbsp;&nbsp;&nbsp;&nbsp; 
$\frac{\delta z H^\Omega}{\delta t} = - \frac{\delta z F^\Omega}{\delta z},$

&nbsp;&nbsp;&nbsp;&nbsp; 
$\frac{\delta z H^\Omega}{\delta t} = - F^\Omega.$

Discretizing the time interval with $\Delta t$ and layer depth with $\Delta z$:

&nbsp;&nbsp;&nbsp;&nbsp; 
$\Delta z H^\Omega = - F^\Omega \Delta t.$

Defining $U$ `[J m-2]` as the normalized enthalpy $H$ `[J m-3]` over layer depth $z$ `[m]`:

&nbsp;&nbsp;&nbsp;&nbsp; 
$\Delta U^\Omega = -F^\Omega\Delta t$

For a given domain $\Omega$, energy conservation is given by:

&nbsp;&nbsp;&nbsp;&nbsp; 
$\Delta U = \left[U^{n+1} - U^{n}\right] + F^{n+1}\Delta t = 0,$

where:

&nbsp;&nbsp;&nbsp;&nbsp; 
$U^{n+1} = \Sigma_i [ H^{n+1}_i z_i^n],$

and $F^{n+1}$ is the net average flux across all layers `i`.

## Assumptions
The number and of distribution of layers in the snow and soil domain is constant through time. This is part of the experimental design.

## Meta data

| Data  | Value  |
|:---|:---|
| Model name| Structure for Unifying Multiple Modelling Alternatives (SUMMA) |
| Model version  | See attributes in output .nc file |
| Model reference | Clark et al. (2015a,b) |
| Model runs by | R. Zolfaghari |
| Notebook code by | W. Knoben |

In [11]:
# modules
import numpy as np
import pandas as pd
import xarray as xr
from operator import truediv 

In [12]:
# Specify the data locations relative to the notebook
file_paths = {'Colbeck exp. 1'     : '../lt5_colbeck1976/output/colbeck1976-exp1_output_timestep.nc',
              'Colbeck exp. 2'     : '../lt5_colbeck1976/output/colbeck1976-exp2_output_timestep.nc',
              'Colbeck exp. 3'     : '../lt5_colbeck1976/output/colbeck1976-exp3_output_timestep.nc',
              'Mizoguchi'          : '../lt6_mizoguchi1990/output/mizoguchi1990_output_timestep.nc'}

In [13]:
# Selection parameters
# We can use these to remove the GRU and HRU dimensions from the output files > easier data handling
GRU, HRU = 0,0

In [14]:
# function to extract enthalpy values
def calc_eb(dat,domain):
    
    # Make some storage variables
    vec_nrgError = []
    vec_enthalpy = [] 
    vec_enthalpyDiff = []
    
    # Find the timestep size [s]
    dt = round((dat['time'][1] - dat['time'][0]).values/np.timedelta64(1, 's'))
    if printFlag: print('    Time step size = {} s.'.format(dt))
    
    # Start of time loop
    for t in range(0,len(dat['time'])-2):
    
        # Find the layers we're interested in
        if domain == 'soil':
            
            # Soil layers start where the snow pack ends. With Python's 0-based indexing, 
            # the number of snow layers is the index at which we find the first soil layer.
            layerTop    = dat['nSnow'].isel(time=t).values.astype('int') 
            layerBottom = dat['nLayers'].isel(time=t).values.astype('int') 
            
        elif domain == 'snow':
            layerTop    = 0 # By definition 
            
            # We'll be using slice(0,nSnow). In this the last index is not returned so we 
            # get indices [0,nSnow-1]
            layerBottom = dat['nSnow'].isel(time=t).values.astype('int')      
        if printFlag: print('    Extracting layers {} to {}'.format(layerTop,layerBottom))
        
        # Get the layer variables
        layerDpth0 = dat['mLayerDepth'].isel(   time=t,   midToto=slice(layerTop,layerBottom)).values # [m]
        layerDpth1 = dat['mLayerDepth'].isel(   time=t+1, midToto=slice(layerTop,layerBottom)).values # [m]
        layerEnth1 = dat['mLayerEnthalpy'].isel(time=t+1, midToto=slice(layerTop,layerBottom)).values # [J m-3]
        layerEnth2 = dat['mLayerEnthalpy'].isel(time=t+2, midToto=slice(layerTop,layerBottom)).values # [J m-3]
        layerFluxI = dat['iLayerNrgFlux'].isel( time=t+2, ifcToto=layerTop).values #          [W m-2] = [J s-1 m-2]
        layerFluxO = dat['iLayerNrgFlux'].isel( time=t+2, ifcToto=layerBottom).values #       [W m-2] = [J s-1 m-2]

        if printFlag: print('    Layer values')
        if printFlag: print('        Dpth0 = {}'.format(layerDpth0))
        if printFlag: print('        Dpth1 = {}'.format(layerDpth1))
        if printFlag: print('        Enth1 = {}'.format(layerEnth1))
        if printFlag: print('        Enth2 = {}'.format(layerEnth2))
        if printFlag: print('        FluxI = {}'.format(layerFluxI))
        if printFlag: print('        FluxO = {}'.format(layerFluxO))
        
        # Do the computation
        U0 = sum(layerEnth1 * layerDpth0)       # [J m-3] * [m]     = [J m-2]
        U1 = sum(layerEnth2 * layerDpth1)       # [J m-3] * [m]     = [J m-2]
        Fn = -1* (layerFluxI - layerFluxO) * dt # [J s-1 m-2] * [s] = [J m-2]
        
        if printFlag: print('    Energy values')
        if printFlag: print('        U0 = {}'.format(U0))
        if printFlag: print('        U1 = {}'.format(U1))
        if printFlag: print('        Fn = {}'.format(Fn))
        
        # Update the outputs
        vec_enthalpy.append(U1)
        vec_enthalpyDiff.append(U1-U0)
        vec_nrgError.append(U1-U0 + Fn)
        
    return vec_nrgError, vec_enthalpy, vec_enthalpyDiff

In [15]:
def calc_relErrSeries(ebe,dStates,states):
    
    # Convert to Numpy arrays for easier indexing
    ebe = np.array(ebe)
    dStates = np.array(dStates)
    states = np.array(states)
    
    # Calculate the relative error series
    rn = list(map(truediv,  ebe[np.nonzero(dStates)[0]], dStates[np.nonzero(dStates)[0]])) # divide two lists element-wise
    rrn = list(map(truediv, ebe[np.nonzero(states)[0]],  states[np.nonzero(states)[0]])) 

    return rn, rrn

Processing starts here.

In [19]:
calculateRelativeMetrics = False

In [20]:
# initiate some lists
test_name   = []
maxAbsErr   = []
meanAbsErr  = []
cumAbsErr   = []
maxRelErr1  = []
meanRelErr1 = []
maxRelErr2  = []
meanRelErr2 = []

# loop over the files
printFlag = False
for test,file in file_paths.items():
    
    # progress
    print('Working on ' + test)
    
    # load the data
    if 'Colbeck' in test:
        dat = xr.open_dataset( file ).isel(hru=HRU).load() # Colbeck output has no GRU dimensions
        domain = 'snow'
    else: # i.e. Mizoguchi
        dat = xr.open_dataset( file ).isel(hru=HRU, gru=GRU).load()
        domain = 'soil'
        
    # Get energy balance values
    ebe,states,dStates = calc_eb(dat,domain)
         
    # Store the remaining metrics
    test_name.append(test)
    maxAbsErr.append(np.max(np.abs(ebe)))
    meanAbsErr.append(np.mean(np.abs(ebe)))
    cumAbsErr.append(np.sum(np.abs(ebe)))
    
    # Calculate the relative error metrics if requested
    if calculateRelativeMetrics:
        rn, rrn = calc_relErrSeries(ebe,dStates,states)
        maxRelErr1.append(np.max(rn))
        meanRelErr1.append(np.mean(rn))      
        maxRelErr2.append(np.max(rrn))
        meanRelErr2.append(np.mean(rrn))

Working on Colbeck exp. 1
Working on Colbeck exp. 2
Working on Colbeck exp. 3
Working on Mizoguchi


In [21]:
# Make a dataframe
if not calculateRelativeMetrics:
    results = pd.DataFrame( {'Test'                   : test_name,
                             'Max. abs. err. [J m-2]' : maxAbsErr,
                             'Mean abs. err. [J m-2]' : meanAbsErr,
                             'Cum. abs. err. [J m-2]' : cumAbsErr},
                           columns = ['Test', \
                                      'Max. abs. err. [J m-2]', \
                                      'Mean abs. err. [J m-2]', \
                                      'Cum. abs. err. [J m-2]'])
    
else:
    results = pd.DataFrame( {'Test'                   : test_name,
                             'Max. abs. err.   [J m-2]' : maxAbsErr,
                             'Mean abs. err.   [J m-2]' : meanAbsErr,
                             'Cum. abs. err.   [J m-2]' : cumAbsErr,
                             'Max. rel. err. 1 [-]'     : maxRelErr1,
                             'Mean rel. err. 1 [-]'     : meanRelErr1,
                             'Max. rel. err. 2 [-]'     : maxRelErr2,
                             'Mean rel. err. 2 [-]'     : meanRelErr2},
                           columns = ['Test', \
                                      'Max. abs. err.   [J m-2]', \
                                      'Mean abs. err.   [J m-2]', \
                                      'Cum. abs. err.   [J m-2]', \
                                      'Max. rel. err. 1 [-]', \
                                      'Mean rel. err. 1 [-]', \
                                      'Max. rel. err. 2 [-]', \
                                      'Mean rel. err. 2 [-]'])

results

,Test,Max. abs. err. [J m-2],Mean abs. err. [J m-2],Cum. abs. err. [J m-2]
0,Colbeck exp. 1,0.027277,0.001811,1.083023
1,Colbeck exp. 2,0.047085,0.007823,4.677996
2,Colbeck exp. 3,0.075294,0.018830,11.260440
3,Mizoguchi,16.603020,0.012181,43.826041
